In [1]:
pwd


'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 312 kB 17.9 MB/s eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |█████████████████████████

In [3]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D 
from keras.preprocessing.image import ImageDataGenerator


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-pack

In [4]:
tensorflow.__version__

'1.14.0'

In [5]:
tensorflow.keras.__version__

'2.2.4-tf'

# image data argumentation

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_a7065add3d834562bf7e62228951934b = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_a7065add3d834562bf7e62228951934b = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_a7065add3d834562bf7e62228951934b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='rPZ-p0RRWxZBKedxEbuybERq4hIlC6KyAIfENXj4SZL1',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_a7065add3d834562bf7e62228951934b)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_1 = client_a7065add3d834562bf7e62228951934b.get_object(Bucket='imageclassification-donotdelete-pr-gqtgizzimr2slj', Key='Dataset.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_1, "__iter__"): streaming_body_1.__iter__ = types.MethodType( __iter__, streaming_body_1 ) 


In [8]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
pwd

'/home/wsuser/work'

In [10]:
import os
filenames = os.listdir('/home/wsuser/work/Dataset/train')

# Loading our data and performing data agumentation

In [11]:
x_train = train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/train',target_size = (64,64),batch_size = 5, color_mode='grayscale', class_mode = "categorical")
x_test= test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test',target_size = (64,64),batch_size = 5, color_mode = 'grayscale',class_mode = "categorical" )

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [12]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [13]:
print(x_test.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


In [14]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 99, 1: 99, 2: 99, 3: 99, 4: 99, 5: 99})

# creating the model
# Adding CNN layers

In [108]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())





# Adding dense layers

In [109]:
# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax')) # softmax for more than 2

In [110]:
classifier.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_7 (Dense)              (None, 6)                

# compiling the model

In [111]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model


# fitting the model

In [112]:
 classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10
119/119 [==============================] - 8s 66ms/step - loss: 1.4068 - acc: 0.4158 - val_loss: 0.7995 - val_acc: 0.6667
Epoch 2/10
119/119 [==============================] - 7s 58ms/step - loss: 0.6894 - acc: 0.7222 - val_loss: 0.7428 - val_acc: 0.7000
Epoch 3/10
119/119 [==============================] - 7s 58ms/step - loss: 0.4567 - acc: 0.8114 - val_loss: 0.5632 - val_acc: 0.8333
Epoch 4/10
119/119 [==============================] - 7s 55ms/step - loss: 0.3763 - acc: 0.8519 - val_loss: 0.4049 - val_acc: 0.7667
Epoch 5/10
119/119 [==============================] - 7s 57ms/step - loss: 0.3233 - acc: 0.8771 - val_loss: 0.4402 - val_acc: 0.8667
Epoch 6/10
119/119 [==============================] - 7s 56ms/step - loss: 0.2515 - acc: 0.9057 - val_loss: 0.2252 - val_acc: 0.9000
Epoch 7/10
119/119 [==============================] - ETA: 0s - loss: 0.2155 - acc: 0.919 - 7s 56ms/step - loss: 0.2175 - acc: 0.9192 - val_loss: 0.2187 - val_acc: 0.9667
Epoch 8/10
119/119 [===========

# saving our model

In [126]:
classifier.save('gesture.h5')

In [127]:
!tar -zcvf image-classification-model_new.tgz gesture.h5

gesture.h5


In [128]:
ls -1

Dataset/
gesture.h5
image-classification-model_new.tgz
model.tar.ga
model.tar.gc


In [129]:
!pip install watson-machine-learning-client --upgrade

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Requirement already up-to-date: watson-machine-learning-client in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (1.0.391)


In [130]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com",
                "apikey":"GU8qMKqu1lH6gyiwgXBKldXeW1u02OFay1loLjVLGbcJ"}
client = APIClient(wml_credentials)

In [131]:
client = APIClient(wml_credentials)

In [132]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [133]:
space_uid=guid_space_name(client,'imageclassification')
print("space UID = "+space_uid)

space UID = 87e990a9-c2f7-438a-9467-1b3f5b5a7076


In [134]:
client.set.default_space(space_uid)

'SUCCESS'

In [135]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029a-cc97-56da-b8e0-39c3880dbbe7  base
tensorflow_2.1-py3.6           1eb25b8

In [136]:
software_space_uid=client.software_specifications.get_uid_by_name("default_py3.7")
software_space_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [137]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"Image processing and model building",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
                                            )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [138]:
model_id

'2199cd47-e0ac-4d9f-991f-ad314f41b7e0'

In [139]:
client.repository.download(model_id,'model.tar.gb')

Successfully saved model content to file: 'model.tar.gb'


'/home/wsuser/work/model.tar.gb'

In [140]:
import tensorflow as tf
from tensorflow import keras
import h5py

# Initialization

loaded_model = keras.models.load_model('gesture.h5')

In [141]:
from keras.preprocessing import image

In [142]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_a7065add3d834562bf7e62228951934b.get_object(Bucket='imageclassification-donotdelete-pr-gqtgizzimr2slj', Key='3.jpg')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 


In [143]:
img = image.load_img(streaming_body_2,target_size = (64,64) )

In [144]:
import numpy as np
x=image.img_to_array(img)
x =np.expand_dims(x,axis=0)

# redicting

In [104]:
pred=classifier.predict_classes(x)

In [105]:
pred[0]

5

In [107]:
index=['0','1','2','3','4','5']
result=str(index[pred[0]])
result

'5'